In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("../data/llamadas_emargencia.csv", sep=";", encoding='latin-1')
df = df[df["ANIO"]==2023]
df_g = df.groupby(["TIPO_DETALLE"])[["CANT_INCIDENTES"]].sum()
df_g["PCT"] = 100 * df_g["CANT_INCIDENTES"]/df_g["CANT_INCIDENTES"].sum()
df_g = df_g.sort_values(by="CANT_INCIDENTES", ascending=False)
df_g

C:\Users\crisi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,CANT_INCIDENTES,PCT
TIPO_DETALLE,,
RIÑA,320486,18.406179
RUIDO,217835,12.510718
ACCIDENTE DE TRÁNSITO,151179,8.682525
VERIFICAR SITUACIÓN,97243,5.584868
MALTRATO,87527,5.026858
...,...,...
EVENTO NATURAL,178,0.010223
ACCIÓN SUBVERSIVA,20,0.001149
ACCIDENTE DE AVIACIÓN,4,0.000230
